In [1]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00


In [2]:
import torch
import random
import shutil
from datasets import Dataset as HF_Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    pipeline
)
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bert_score


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

input_file_path = "/kaggle/input/dataset/conv_data/therapy_train/data-00000-of-00001.arrow"
writable_file_path = "/kaggle/working/train_data.arrow"
val_data_load_path = "/kaggle/input/dataset/conv_data/therapy_val/data-00000-of-00001.arrow"
val_data_write_path = "/kaggle/working/val_data.arrow"

shutil.copy(input_file_path, writable_file_path)
shutil.copy(val_data_load_path, val_data_write_path)



'/kaggle/working/val_data.arrow'

In [3]:
import torch
from torch.utils.data import DataLoader
from datasets import Dataset as HF_Dataset, load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    pipeline,
    DataCollatorForSeq2Seq
)
import numpy as np
from accelerate import Accelerator


accelerator = Accelerator()
DEVICE = accelerator.device

import pyarrow as pa
from datasets import Dataset


import torch
import pyarrow as pa
from datasets import Dataset, load_dataset


def load_any_file(path):
    
    try:
        
        table = pa.ipc.open_file(pa.memory_map(path, 'r')).read_all()
        return Dataset.from_arrow(table)
    except pa.ArrowInvalid:
        try:
            table = pa.parquet.read_table(path)
            return Dataset.from_arrow(table)
        except:
            return load_dataset('json', data_files=path)['train']
    except Exception as e:
        raise ValueError(f"Unsupported file format for {path}: {str(e)}")


def load_data_with_validation(input_path, output_path):
    try:
        dataset = load_any_file(input_path)
        return dataset
    except Exception as e:

        print(f"Error loading {input_path}: {e}, trying HF load_dataset")
        return load_dataset('arrow', data_files=input_path)['train']


dataset = load_data_with_validation(input_file_path, writable_file_path)
val_dataset = load_data_with_validation(val_data_load_path, val_data_write_path)


dataset = dataset.with_format("torch", device=DEVICE)
val_dataset = val_dataset.with_format("torch", device=DEVICE)

def build_triples_batched(dataset):
    
    def process_batch(batch):
        
        batch_size = len(batch["input_text"])
        return {
            
            "history": batch["input_text"][:-1],
            "ut": batch["target_text"][:-1],
            "next_input": batch["input_text"][1:batch_size]
        }
    
    processed = dataset.map(
        process_batch,
        batched=True,
        batch_size=1024,
        remove_columns=dataset.column_names
    )
    
    def extract_ut1(batch):
        return {
            "ut1": [
                next(
                    (seg.split("P:")[1].strip() for seg in text.split("[SEP]") if "P:" in seg),
                    None
                )
                for text in batch["next_input"]
            ]
        }
    
    return processed.map(
        extract_ut1,
        batched=True,
        batch_size=1024
    ).filter(lambda x: x["ut1"] is not None)

train_triples = build_triples_batched(dataset)
val_pairs = val_dataset.map(
    lambda x: {"history": x["input_text"], "ut": x["target_text"]},
    batched=True,
    batch_size=1024
)

Generating train split: 0 examples [00:00, ? examples/s]

Failed to load JSON from file '/kaggle/input/dataset/conv_data/therapy_train/data-00000-of-00001.arrow' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Invalid value. in row 0


Error loading /kaggle/input/dataset/conv_data/therapy_train/data-00000-of-00001.arrow: An error occurred while generating the dataset, trying HF load_dataset


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4008 [00:00<?, ? examples/s]

Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4004 [00:00<?, ? examples/s]

Map:   0%|          | 0/576 [00:00<?, ? examples/s]

In [4]:


BASE_MODEL = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)


generator_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16).to(DEVICE)
simulator_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16).to(DEVICE)
print("models loaded")


data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8)  
print("data collaor done")

def tokenize_function(batch):
    
    generator_inputs = tokenizer(
        batch["history"],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Generator targets
    with tokenizer.as_target_tokenizer():
        generator_targets = tokenizer(
            batch["ut"],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    
    # Simulator inputs
    simulator_texts = [f"{h} [SEP] T: {ut}" for h, ut in zip(batch["history"], batch["ut"])]
    simulator_inputs = tokenizer(
        simulator_texts,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Simulator targets
    with tokenizer.as_target_tokenizer():
        simulator_targets = tokenizer(
            batch["ut1"],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    
    return {
        "generator_input_ids": generator_inputs.input_ids,
        "generator_attention_mask": generator_inputs.attention_mask,
        "generator_labels": generator_targets.input_ids,
        "simulator_input_ids": simulator_inputs.input_ids,
        "simulator_attention_mask": simulator_inputs.attention_mask,
        "simulator_labels": simulator_targets.input_ids,
    }


print("create optimized dataloaders")
train_dataset = train_triples.map(
    tokenize_function,
    batched=True,
    batch_size=1024,
    remove_columns=train_triples.column_names
)
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    collate_fn=data_collator,
    pin_memory=True,
    num_workers=4
)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

models loaded
data collaor done
create optimized dataloaders


Map:   0%|          | 0/4001 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
# tokenize prparation
def tokenize_generator(batch):
    
    inputs = tokenizer(
        batch["history"],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["ut"],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": labels.input_ids
    }

def tokenize_simulator(batch):
    
    inputs = tokenizer(
        [f"{h} [SEP] T: {ut}" for h, ut in zip(batch["history"], batch["ut"])],
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["ut1"],
            max_length=128,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": labels.input_ids
    }


generator_train_dataset = train_triples.map(
    tokenize_generator,
    batched=True,
    batch_size=1024,
    remove_columns=train_triples.column_names
)

simulator_train_dataset = train_triples.map(
    tokenize_simulator,
    batched=True,
    batch_size=1024,
    remove_columns=train_triples.column_names
)


generator_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
simulator_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [8]:


def train_model(model, tokenizer, dataset, output_dir):
    
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=3,
        predict_with_generate=True,
        fp16=torch.cuda.is_available(),  
        dataloader_num_workers=4,
        optim="adamw_torch",
        report_to="none",
        logging_steps=100,
        save_strategy="steps",
        save_steps=500,
        max_grad_norm=1.0  
    )

    
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        pad_to_multiple_of=8,
        padding=True,
        return_tensors="pt"
    )

    
    model = model.to(DEVICE, dtype=torch.float32)

    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("Starting training...")
    trainer.train()
    return model


generator_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
simulator_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# # Train models
print("Training Generator Model...")
generator_model = train_model(generator_model, tokenizer, generator_train_dataset, "generator")

print("\nTraining Simulator Model...")
simulator_model = train_model(simulator_model, tokenizer, simulator_train_dataset, "simulator")

<ipython-input-8-7da89d3d7065>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Starting training...


/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting

Step,Training Loss
100,5.558600
200,1.054400
300,0.900600


/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting


Training Simulator Model...
Starting training...


<ipython-input-8-7da89d3d7065>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], d

Step,Training Loss
100,5.510300
200,0.248700
300,0.164700


/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting

In [25]:
# !zip -r saved_model.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/val_data.arrow (deflated 96%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/generator/ (stored 0%)
  adding: kaggle/working/generator/checkpoint-375/ (stored 0%)
  adding: kaggle/working/generator/checkpoint-375/training_args.bin (deflated 52%)
  adding: kaggle/working/generator/checkpoint-375/config.json (deflated 62%)
  adding: kaggle/working/generator/checkpoint-375/tokenizer_config.json (deflated 95%)
  adding: kaggle/working/generator/checkpoint-375/trainer_state.json (deflated 58%)
  adding: kaggle/working/generator/checkpoint-375/scheduler.pt (deflated 56%)
  adding: kaggle/working/generator/checkpoint-375/optimizer.pt (deflated 7%)
  adding: kaggle/working/generator/checkpoint-375/generation_config.json (deflated 29%)
  adding: kaggle/working/generator/checkpoint-375/tokenizer.json (deflated 74%)
  adding: kaggle/working/generator/checkpoint-375/model.safetensors (deflated 11%)
  

In [19]:
print(len(train_triples))
print(train_triples[0])

4001
{'history': 'T: Hi you how to do it today? [SEP] P: Great. How are you?', 'ut': "I'm doing well. Thanks for asking.", 'next_input': "T: Hi you how to do it today? [SEP] P: Great. How are you? [SEP] T: I'm doing well. Thanks for asking.", 'ut1': 'Great. How are you?'}


In [48]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from torch.utils.data import DataLoader
from datasets import Dataset
from tqdm import tqdm


emotion_classifier = pipeline(
    "text-classification",
    model="bhadresh-savani/bert-base-uncased-emotion",
    device=0 if torch.cuda.is_available() else -1
)

class TherapeuticTrainer:
    def __init__(self, generator, simulator, tokenizer, emotion_classifier):
        self.generator = generator.to(DEVICE)
        self.simulator = simulator.to(DEVICE)
        self.tokenizer = tokenizer
        self.emotion_classifier = emotion_classifier
        self.data_collator = DataCollatorForSeq2Seq(tokenizer)
        
        
        for param in self.simulator.parameters():
            param.requires_grad = False
            
        self.optimizer = torch.optim.AdamW(self.generator.parameters(), lr=3e-5)
        self.scaler = torch.cuda.amp.GradScaler()

    def calculate_reward(self, responses):
        
        batch_results = self.emotion_classifier(responses)
        positive_emotions = {'joy', 'love', 'surprise'}
        
        
        if isinstance(batch_results[0], dict):
            batch_results = [batch_results]
            
        rewards = []
        for result in batch_results:
            if isinstance(result, list):
                
                score = sum(r['score'] if r['label'] in positive_emotions else -r['score'] for r in result)
            else:
                
                score = 0.0
            rewards.append(score)
            
        return torch.tensor(rewards, device=DEVICE)
    def train_epoch(self, dataloader, num_candidates=5):
        self.generator.train()
        total_loss = torch.tensor(0.0, device=DEVICE)  
        num_batches = len(dataloader)
        
        for batch in tqdm(dataloader):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            
            with torch.cuda.amp.autocast():
                
                outputs = self.generator(**batch)
                sup_loss = outputs.loss
                
                
                gen_outputs = self.generator.generate(
                    batch['input_ids'],
                    max_length=128,
                    num_beams=num_candidates,
                    num_return_sequences=num_candidates,
                    early_stopping=True
                )
                
            
            candidates = self.tokenizer.batch_decode(gen_outputs, skip_special_tokens=True)
            original_inputs = self.tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)
            
            
            sim_inputs = [f"{inp} [SEP] T: {cand}" 
                        for inp, cand in zip(original_inputs * num_candidates, candidates)]
            
            with torch.no_grad():
                sim_tokenized = self.tokenizer(
                    sim_inputs, 
                    return_tensors='pt', 
                    padding=True, 
                    truncation=True,
                    max_length=128
                ).to(DEVICE)
                
                sim_outputs = self.simulator.generate(
                    input_ids=sim_tokenized.input_ids,
                    attention_mask=sim_tokenized.attention_mask,
                    max_length=128,
                    num_beams=3,
                    early_stopping=True
                )
                simulated_responses = self.tokenizer.batch_decode(sim_outputs, skip_special_tokens=True)
            
            
            rewards = torch.tensor([self.calculate_reward(r) for r in simulated_responses], 
                                 device=DEVICE).view(-1, num_candidates)
            
            # feedback learning is happeninig here
            with torch.cuda.amp.autocast():
                # Get dimensions
                batch_size, seq_length, vocab_size = outputs.logits.shape
                
                # Expand log_probs for candidates
                log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)
                log_probs = log_probs.repeat_interleave(num_candidates, dim=0)
                
                # Expand rewards to match log_probs dimensions
                expanded_rewards = rewards.view(-1, 1, 1).expand(
                    -1, seq_length, vocab_size
                )
                
                rl_loss = -torch.mean(log_probs * expanded_rewards)
            
            
            batch_loss = 0.7 * sup_loss + 0.3 * rl_loss
            total_loss += batch_loss  # Accumulate as tensor
            
            
            self.optimizer.zero_grad()
            self.scaler.scale(batch_loss).backward()
            torch.nn.utils.clip_grad_norm_(self.generator.parameters(), 1.0)
            self.scaler.step(self.optimizer)
            self.scaler.update()
        
        # Calculate average loss
        avg_loss = total_loss / num_batches
        return avg_loss.item()  

    


tokenizer = AutoTokenizer.from_pretrained("t5-small")


data_collator = DataCollatorForSeq2Seq(tokenizer)
train_loader = DataLoader(
    generator_train_dataset,
    batch_size=16,
    collate_fn=data_collator,
    shuffle=True
)


trainer = TherapeuticTrainer(
    generator_model,
    simulator_model,
    tokenizer,
    emotion_classifier
)




for epoch in range(3):
    epoch_loss = trainer.train_epoch(train_loader)
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")


trainer.generator.save_pretrained("/kaggle/working/final_therapeutic_model")
tokenizer.save_pretrained("/kaggle/working/final_therapeutic_model")
print("model_saved_successfully")


Device set to use cuda:0
<ipython-input-48-266de778e463>:27: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/225 [00:00<?, ?it/s]<ipython-input-48-266de778e463>:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-48-266de778e463>:102: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 225/225 [27:51<00:00,  7.43s/it]


Epoch 1 Loss: 4.3705
model_saved_successfully


In [49]:
# Training loop
for epoch in range(1,3):
    epoch_loss = trainer.train_epoch(train_loader)
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")


trainer.generator.save_pretrained("/kaggle/working/final2_therapeutic_model")
tokenizer.save_pretrained("/kaggle/working/final2_therapeutic_model")
print("model_saved_successfully")

  0%|          | 0/225 [00:00<?, ?it/s]<ipython-input-48-266de778e463>:57: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-48-266de778e463>:102: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 225/225 [28:27<00:00,  7.59s/it]


Epoch 2 Loss: 4.3438


100%|██████████| 225/225 [27:58<00:00,  7.46s/it]


Epoch 3 Loss: 4.3269
model_saved_successfully


In [34]:
 
def therapeutic_response(context, max_length=128):
    device="cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(
        context,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        add_special_tokens=True  
    ).to(device)
    
    outputs = generator_model.generate(
        inputs.input_ids,
        max_length=max_length,
        min_length=10,
        num_beams=5,
        temperature=0.9,
        early_stopping=True
    )
    
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded if len(decoded) > 5 else "[SAFETY] Could you elaborate?"



test_dialogue = "T: So, alright, let's take a step back and talk about competency. What education did you need for the job you have now? [SEP] P: I have my bachelor's degree. And we got a little bit of training. You know, every so often we get some training at work and stuff like that. [SEP]T: Are you evaluated at work by anybody to see if you re in a job you should be? [SEP] P: Yeah,I have a supervisor so they check up on stuff and also, like, if I feel like I have questions and stuff like that, I can go to them as well."
print(f"Prediction is: {therapeutic_response(test_dialogue)}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Prediction is: Is that a job you're in? Is that a job you're in?


In [58]:
from nltk.translate.bleu_score import corpus_bleu
from bert_score import score as bert_score
from datasets import load_from_disk
import numpy as np

def evaluate_on_test_data(model_path, test_data_path, num_samples=None):
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(DEVICE)
    
    
    test_dataset = load_from_disk(test_data_path)
    
    
    if num_samples:
        test_dataset = test_dataset.select(range(num_samples))
    
    
    references = []
    candidates = []
    
    print(f"Evaluating on {len(test_dataset)} samples...")
    
    for example in test_dataset:
        
        context = example["input_text"]
        generated = generate_response(model, tokenizer, context)
        reference = example["target_text"]
        
        
        references.append([reference.split()])  
        candidates.append(generated.split())
    
    
    bleu = corpus_bleu(references, candidates)
    
    
    cand_strings = [" ".join(c) for c in candidates]
    ref_strings = [" ".join(r[0]) for r in references]
    P, R, F1 = bert_score(cand_strings, ref_strings, lang="en", device=DEVICE)
    
    return {
        "bleu": bleu,
        "bert_precision": np.mean(P.numpy()),
        "bert_recall": np.mean(R.numpy()),
        "bert_f1": np.mean(F1.numpy()),
        "sample_input": context,
        "sample_reference": reference,
        "sample_generated": generated
    }


def generate_response(model, tokenizer, context, max_length=128):
    inputs = tokenizer(
        context,
        return_tensors="pt",
        max_length=128,
        truncation=True
    ).to(DEVICE)
    
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_beams=5,
        early_stopping=True
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Usage example
results = evaluate_on_test_data(
    model_path="/kaggle/working/optimized_therapeutic_model",
    test_data_path="/kaggle/input/dataset/conv_data/therapy_test"  
)

print(f"BLEU Score: {results['bleu']:.4f}")
print(f"BERT F1 Score: {results['bert_f1']:.4f}")
print(f"BERT Precision Score: {results['bert_precision']:.4f}")
print(f"BERT recall Score: {results['bert_recall']:.4f}")


BLEU Score: 0.0253
BERT F1 Score: 0.8151
BERT Precision Score: 0.8154
BERT recall Score: 0.8157


In [59]:
# load model
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
model_path="/kaggle/working/final2_therapeutic_model"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(DEVICE)